# Full Example
THe idea with this NB is to do the same as the pymc_prophet_example NB but do everything (holidays, regressors etc...) in a single example and compare pymc_prohet with prophet. Then we will use this to build the Bayesian PyMC component.

In [2]:
import pandas as pd
from pymcprophet.model import BayesTS, BayesTSConfig
from prophet import Prophet


Importing plotly failed. Interactive plots will not work.


In [ ]:
df = df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df['ds'] = pd.to_datetime(df['ds'])

## Holidays
These are user defined. We will also add auto US ones.

In [ ]:
playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                        '2010-01-24', '2010-02-07', '2011-01-08',
                        '2013-01-12', '2014-01-12', '2014-01-19',
                        '2014-02-02', '2015-01-11', '2016-01-17',
                        '2016-01-24', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((playoffs, superbowls))

## Regressor
Custom dataset for Sundays during the NFL Season.

In [ ]:
def nfl_sunday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6 and (date.month > 8 or date.month < 2):
        return 1
    else:
        return 0

df['nfl_sunday'] = df['ds'].apply(nfl_sunday)

## Modelling

In [11]:
train_df = df.query("ds.dt.year <= 2014")
test_df = df.query("ds.dt.year > 2014")



ts_config = BayesTSConfig()
ts_model = BayesTS(ts_config)
ts_model.add_holiday_country("US")
ts_model.add_holiday(holidays)
ts_model.add_regressor("nfl_sunday")

ts_model.assign_model_matrix(train_df)

ts_model.model_df

,ds,y,nfl_sunday,y_trans,t,t_seasonality,yearly_sin_1,yearly_sin_2,yearly_sin_3,yearly_sin_4,...,NewYearsDayobserved,JuneteenthNationalIndependenceDayobserved,ChristmasDayobserved,VeteransDayobserved,weekly_sin_1,weekly_sin_2,weekly_sin_3,weekly_cos_1,weekly_cos_2,weekly_cos_3
0,2007-12-10,9.590761,0,0.746552,0.000000,13857.0,-0.377462,-0.699079,-0.917267,-0.999745,...,0,0,0,0,-4.338837e-01,7.818315e-01,-9.749279e-01,-0.900969,0.623490,-0.222521
1,2007-12-11,8.519590,0,0.663171,0.000388,13858.0,-0.361478,-0.674069,-0.895501,-0.995827,...,0,0,0,0,-9.749279e-01,4.338837e-01,7.818315e-01,-0.222521,-0.900969,0.623490
2,2007-12-12,8.183677,0,0.637023,0.000776,13859.0,-0.345386,-0.648262,-0.871351,-0.987196,...,0,0,0,0,-7.818315e-01,-9.749279e-01,-4.338837e-01,0.623490,-0.222521,-0.900969
3,2007-12-13,8.072467,0,0.628367,0.001164,13860.0,-0.329192,-0.621687,-0.844881,-0.973892,...,0,0,0,0,5.505235e-14,1.101047e-13,1.984146e-12,1.000000,1.000000,1.000000
4,2007-12-14,7.893572,0,0.614441,0.001552,13861.0,-0.312900,-0.594376,-0.816160,-0.955979,...,0,0,0,0,7.818315e-01,9.749279e-01,4.338837e-01,0.623490,-0.222521,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517,2014-12-27,7.814400,0,0.608278,0.998448,16431.0,-0.090190,-0.179645,-0.267636,-0.353445,...,0,0,0,0,9.749279e-01,-4.338837e-01,-7.818315e-01,-0.222521,-0.900969,0.623490
2518,2014-12-28,8.387995,1,0.652928,0.998836,16432.0,-0.073045,-0.145700,-0.217577,-0.288291,...,0,0,0,0,4.338837e-01,-7.818315e-01,9.749279e-01,-0.900969,0.623490,-0.222521
2519,2014-12-29,8.748146,0,0.680962,0.999224,16433.0,-0.055879,-0.111583,-0.166938,-0.221772,...,0,0,0,0,-4.338837e-01,7.818315e-01,-9.749279e-01,-0.900969,0.623490,-0.222521
2520,2014-12-30,8.312871,0,0.647080,0.999612,16434.0,-0.038696,-0.077334,-0.115856,-0.154204,...,0,0,0,0,-9.749279e-01,4.338837e-01,7.818315e-01,-0.222521,-0.900969,0.623490


In [15]:
ts_model.model_spec.get_holiday_feature_cols()

['playoff',
 'superbowl',
 'NewYearsDay',
 'MemorialDay',
 'JuneteenthNationalIndependenceDay',
 'IndependenceDay',
 'IndependenceDayobserved',
 'LaborDay',
 'VeteransDay',
 'Thanksgiving',
 'ChristmasDay',
 'MartinLutherKingJrDay',
 'WashingtonsBirthday',
 'ColumbusDay',
 'NewYearsDayobserved',
 'JuneteenthNationalIndependenceDayobserved',
 'ChristmasDayobserved',
 'VeteransDayobserved']